In [8]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as matCol
from matplotlib.colors import ListedColormap
import os
import sys
import h5py
import time
%matplotlib qt

import warnings
warnings.filterwarnings("ignore")

#calculate fields to plot
home = 'R:/satellite_data/viirs_data/noaa20/'
# hf_composite_database = h5py.File(home+'databases/composite_database.h5','a')


with h5py.File(home+'databases/VIIRS_burn_Scar_database.h5','r') as hf_database:
    timestamps = list(hf_database.keys())
    dataset_names = list(hf_database[timestamps[0]].keys())

    #stack composites here, where latest pixel overwrites all other
    #the oldest pixel should be no more than n days old (start w/8 days)
    #note a granbule does not eqaul a day, i.e. there is over lap, so use
    #time and day to decide which granule is newer in order to overwrite.
    #empty/ non valid pixels should be auto overwritten, i.e. np.nan or x<=-997
    #work backwords from newest
    #once it works try again but cloud and water clear with -998, -997 repectively
    base_image_shape           = (3604, 2354, 3)
    composite_burn_scar_RGB    = np.empty(base_image_shape)
    composite_burn_scar_RGB[:] = np.nan
    
    start, end = -68, -1#*n_days
    for i, timestamp in enumerate(timestamps[start:end]):
        dataset_dict = {}
        for dataset_name in dataset_names:
            dataset_dict[dataset_name] = hf_database[timestamp+'/'+dataset_name][:]

        r_row_idx    = dataset_dict['regrid_row_idx']
        if np.count_nonzero(r_row_idx == -999)/len(r_row_idx.flatten()) < 0.8:
            pass
        else:
            print(i, 'pass')
            continue
        r_col_idx    = dataset_dict['regrid_col_idx']
        fill_val_idx = dataset_dict['fill_val_idx']

        # NBR           = dataset_dict['NBR'          ][r_row_idx, r_col_idx]
        # BRF_RGB       = dataset_dict['BRF_RGB'      ][r_row_idx, r_col_idx]
        cldmsk          = dataset_dict['cldmsk'       ][r_row_idx, r_col_idx]
        burn_scar_RGB   = dataset_dict['burn_scar_RGB'][r_row_idx, r_col_idx]
        land_water_mask = dataset_dict['land_water_mask'][r_row_idx, r_col_idx]
        # lat_regrid    = dataset_dict['lat'][r_row_idx, r_col_idx]
        # lon_regrid    = dataset_dict['lon'][r_row_idx, r_col_idx]
        # R_M7, R_M11 = burn_scar_RGB[:,:,1], burn_scar_RGB[:,:,0]
        # burn_scar_composite = get_burn_scar_composite(R_M7, R_M11, geotiff=False)


        # NBR[fill_val_idx[0], fill_val_idx[1]]           = np.nan
        # BRF_RGB[fill_val_idx[0], fill_val_idx[1]]       = np.nan
        cldmsk[fill_val_idx[0], fill_val_idx[1]]        = np.nan
        burn_scar_RGB[fill_val_idx[0], fill_val_idx[1],:] = np.nan
        # lat_regrid[fill_val_idx[0], fill_val_idx[1]]    = np.nan
        # lon_regrid[fill_val_idx[0], fill_val_idx[1]]    = np.nan

        # NBR[NBR<=-997]           = np.nan
        # BRF_RGB[BRF_RGB<=-997]       = np.nan
        # cldmsk[cldmsk<=-997]        = np.nan
        burn_scar_RGB[burn_scar_RGB==-999] = np.nan

        #cloud and water clear
        burn_scar_RGB[cldmsk < 2] = np.nan #noise from probably clear, may mask it out
        #the probably cloudy is important to keep other wise lose alot to smoke maybe
        burn_scar_RGB[land_water_mask == 0] = np.nan #just take out water (from cloud mask file)

        #combine data by populating nan with cloud free water free data from latest granule
        unpopulated_composite_idx = np.where(np.isnan(composite_burn_scar_RGB))
        unpopulated_current_idx   = np.where(np.isnan(burn_scar_RGB))
        temp_composite            = np.copy(composite_burn_scar_RGB)
        composite_burn_scar_RGB[unpopulated_composite_idx[0], unpopulated_composite_idx[1],:] = \
                             burn_scar_RGB[unpopulated_composite_idx[0], unpopulated_composite_idx[1],:]

        #restore composite pixels overwritten by nan and fill vals -997 to -999
        composite_burn_scar_RGB[unpopulated_current_idx[0], unpopulated_current_idx[1], :] = \
                                        temp_composite[unpopulated_current_idx[0], unpopulated_current_idx[1], :]
        print('{:03d}/64 {} composite updated'.format(i, timestamp))
        
        if i%64<8:
            plt.figure(figsize=(10,15))
            plt.imshow(composite_burn_scar_RGB)
            plt.draw()
            print('plotting {:03d}/64 {} composite updated'.format(i, timestamp))
#         plt.pause(5)
            


0 pass
001/64 2021233.1930_08.21.2021 composite updated


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


plotting 001/64 2021233.1930_08.21.2021 composite updated
002/64 2021233.2106_08.21.2021 composite updated


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


plotting 002/64 2021233.2106_08.21.2021 composite updated
003/64 2021233.2112_08.21.2021 composite updated


...

plotting 007/64 2021234.2054_08.22.2021 composite updated
8 pass
9 pass
10 pass
11 pass
012/64 2021235.2030_08.23.2021 composite updated
13 pass
14 pass
15 pass
16 pass
17 pass
18 pass
19 pass
020/64 2021236.2012_08.24.2021 composite updated
21 pass
22 pass
023/64 2021236.2154_08.24.2021 composite updated
24 pass
25 pass
26 pass
27 pass
28 pass
029/64 2021237.1954_08.25.2021 composite updated
030/64 2021237.2130_08.25.2021 composite updated
031/64 2021237.2136_08.25.2021 composite updated
32 pass
033/64 2021238.1936_08.26.2021 composite updated
034/64 2021238.2112_08.26.2021 composite updated
035/64 2021238.2118_08.26.2021 composite updated
36 pass
037/64 2021239.1918_08.27.2021 composite updated
038/64 2021239.2054_08.27.2021 composite updated
039/64 2021239.2100_08.27.2021 composite updated
40 pass
041/64 2021240.1900_08.28.2021 composite updated
042/64 2021240.2036_08.28.2021 composite updated
043/64 2021240.2042_08.28.2021 composite updated
44 pass
45 pass
46 pass
47 pass
48 pass
0

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


plotting 064/64 2021243.2118_08.31.2021 composite updated
065/64 2021243.2124_08.31.2021 composite updated


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


plotting 065/64 2021243.2124_08.31.2021 composite updated
66 pass


In [ ]:
#         #check if 8 days has been recorded first, which is 64 files
#         if i>=64:
#             #then record the composite as it updates every day, which is 8 files into the database
#             if i%8==0:
#                 R_M7, R_M11 = np.copy(composite_burn_scar_RGB[:,:,1]), np.copy(composite_burn_scar_RGB[:,:,0])
#                 NBR_composite = (R_M11-R_M7)/(R_M11+R_M7)

#                 bad_val = np.nan
#         #         R_M11_ = ndimage.gaussian_filter(R_M11, sigma=1)
#                 R_M11_ = R_M11
#                 R_M11_[R_M7  > 0.2]  = bad_val #for clear no smoke only
#                 R_M11_[R_M7 < 0.0281] = bad_val
#                 R_M11_[R_M11 < 0.06]  = bad_val
#                 # R_M11[R_M11 > 0.3]   = bad_val

#                 composite_burn_scar_thresh = R_M11_
#                 NBR_composite[np.isnan(composite_burn_scar_thresh)] = np.nan

#                 #############################################################
#                 #plot
#                 %matplotlib qt
#                 rows, cols, colors = base_image_shape 
#                 X,Y = np.meshgrid(np.arange(cols), np.arange(rows))
#                 plt.style.use('dark_background')
#                 f, ax = plt.subplots(ncols=3, sharex=True, sharey=True, figsize=(24,13))
#                 # im = plt.imshow(composite_burn_scar_RGB[:,:,1], cmap='jet',vmin=0.0281, vmax=0.1346)
#                 # im.cmap.set_over('grey')
#                 # im.cmap.set_under('cyan')
#                 im = ax[0].imshow(composite_burn_scar_RGB)
#                 ax[1].imshow(composite_burn_scar_RGB)
#                 # ax[1].imshow(composite_burn_scar_thresh, cmap='jet', vmax=0.22)
#                 ax[1].imshow(NBR_composite, cmap='jet', vmax=0.25)
#                 # ax[1].contourf(X,Y,composite_burn_scar_thresh, levels=40, cmap='jet')
#                 ax[2].imshow(2*composite_burn_scar_RGB)
#                 for a in ax.flat:
#                     a.set_xticks([])
#                     a.set_yticks([])
#                 f.suptitle(timestamps[i][-10:], fontsize=24)
#                 plt.tight_layout()
# #                 plt.savefig(home+'databases/'+timestamps[i]+'.jpg')
#                 plt.show()
#                 try:
#                     hf_composite_database.create_dataset(timestamp, data=composite_burn_scar_RGB, compression='gzip')
#                     print('{:03d}/604 {} composite recorded'.format(i, timestamp))
#                 except:
#                     hf_composite_database[timestamp][:] = composite_burn_scar_RGB
#                     print('{:03d}/604 {} composite updated'.format(i, timestamp))
                

# hf_composite_database.close()

In [ ]:
from scipy import ndimage
import matplotlib.pyplot as plt
%matplotlib qt

with h5py.File(home+'databases/VIIRS_burn_Scar_database.h5','r') as hf_composite_database:
    timestamps = list(hf_composite_database.keys())#[-30:-1]
    t_temp = [int(x[8:12]) for x in timestamps]
    print(np.sort(t_temp))
    plt.hist(t_temp)
    plt.show()
#     for i in range(len(timestamps)):
#         pass
#         composite_burn_scar_RGB = hf_composite_database[timestamps[i]][:]
        

#     R_M7, R_M11 = np.copy(composite_burn_scar_RGB[:,:,1]), np.copy(composite_burn_scar_RGB[:,:,0])
#     NBR_composite = (R_M11-R_M7)/(R_M11+R_M7)

#     bad_val = 0
# #         R_M11_ = ndimage.gaussian_filter(R_M11, sigma=1)
#     R_M11[R_M7  > 0.1346] = bad_val #for clear no smoke only
#     R_M11[R_M7  < 0.0281] = bad_val

#     composite_burn_scar_thresh = R_M11
#     composite_burn_scar_thresh = ndimage.gaussian_filter(composite_burn_scar_thresh, sigma=3)
#     composite_burn_scar_thresh[composite_burn_scar_thresh<0.01]=np.nan

#     NBR_composite[np.isnan(composite_burn_scar_thresh)] = np.nan
#     NBR_composite[composite_burn_scar_thresh<=0] = np.nan

#     #############################################################
#     #plot
#     %matplotlib qt
#     rows, cols, colors = base_image_shape 
#     X,Y = np.meshgrid(np.arange(cols), np.arange(rows))
#     plt.style.use('dark_background')
#     f, ax = plt.subplots(ncols=3, sharex=True, sharey=True, figsize=(24,13))
#     # im = plt.imshow(composite_burn_scar_RGB[:,:,1], cmap='jet',vmin=0.0281, vmax=0.1346)
#     # im.cmap.set_over('grey')
#     # im.cmap.set_under('cyan')
#     im = ax[0].imshow(composite_burn_scar_RGB)
#     ax[1].imshow(composite_burn_scar_RGB)
#     # ax[1].imshow(composite_burn_scar_thresh, cmap='jet', vmax=0.22)
#     ax[1].imshow(NBR_composite, cmap='gist_ncar', vmax=0.5)
#     # ax[1].contourf(X,Y,composite_burn_scar_thresh, levels=40, cmap='jet')
#     ax[2].imshow(2*composite_burn_scar_RGB)
#     for a in ax.flat:
#         a.set_xticks([])
#         a.set_yticks([])
# #     f.suptitle(timestamps[i][-10:], fontsize=24)
#     f.suptitle('09.01.2021', fontsize=24)
#     plt.tight_layout()
# #         plt.savefig(home+'databases/'+timestamps[i]+'.jpg')
#     plt.show()
# #         break
#         # plt.close()

In [ ]:
%matplotlib qt
rows, cols, colors = base_image_shape 
X,Y = np.meshgrid(np.arange(cols), np.arange(rows))
plt.style.use('dark_background')
f, ax = plt.subplots(ncols=2, sharex=True, sharey=True, figsize=(24,13))
# im = plt.imshow(composite_burn_scar_RGB[:,:,1], cmap='jet',vmin=0.0281, vmax=0.1346)
# im.cmap.set_over('grey')
# im.cmap.set_under('cyan')
im = ax[0].imshow(2*composite_burn_scar_RGB)
ax[1].imshow(composite_burn_scar_RGB)
# ax[1].imshow(composite_burn_scar_thresh, cmap='jet', vmax=0.22)
ax[1].imshow(NBR_composite, cmap='jet', vmax=0.25)
# ax[1].contourf(X,Y,composite_burn_scar_thresh, levels=40, cmap='jet')
# ax[2].imshow(2*composite_burn_scar_RGB)
for a in ax.flat:
    a.set_xticks([])
    a.set_yticks([])
plt.tight_layout()
plt.show()
# plt.close()